# IRC Behavioral Analysis - Graphs

### Imports

In [2]:
from graphviz import Digraph

### Graph structure

In [3]:


# class Node(object):
#     def __init__(self, name: str):
#         self.name = name

#     def __eq__(self, other):
#         if isinstance(other, Node):
#             return self.name == other.name
#         return False

#     def id(self):
#         hash_node = lambda v: str(abs(hash(v)) % (10 ** 8))
#         return hash_node(self.name)
#         # return self.ip

#     def label(self):
#         return self.name


# class Edge(object):
#     def __init__(self, src_node: Node, dst_node: Node):
#         self.src_node = src_node
#         self.dst_node = dst_node

#     def __eq__(self, other):
#         if isinstance(other, Edge):
#             return self.src_node == other.src_node and self.dst_node == other.dst_node
#         return False


# class MsgEdge(Edge):
#     def __init__(self, src_node: Node, dst_node: Node, msg: str):
#         super().__init__(src_node, dst_node)
#         self.msg = msg

#     def __eq__(self, other):
#         if isinstance(other, MsgEdge):
#             return self.src_node == other.src_node and \
#                    self.dst_node == other.dst_node and \
#                    self.msg == other.msg
#         return False


# class Graph(object):
#     def __init__(self, nodes=None, edges=None):
#         if nodes is None:
#             nodes = []
#         if edges is None:
#             edges = []
#         self.nodes = nodes
#         self.edges = edges

#     def add_node(self, node: Node) -> bool:
#         """ :returns False if the node is already in nodes
#             :returns True otherwise
#         """
#         if node not in self.nodes:
#             self.nodes.append(node)
#             return True

#         return False

#     def add_edge(self, edge: Edge) -> bool:
#         """ :returns False if the edge is already in edges
#             :returns True otherwise
#         """
#         if edge not in self.edges:
#             self.edges.append(edge)
#             return True
#         return False

In [4]:
# def build_graph(logs):
#     print('building graph...')
#     graph = Graph()
#     for log in logs[:100]:
#         v1_source = log['source']
#         v2_target = log['target']

#         v1, v2 = Node(v1_source), Node(v2_target)

#         e = MsgEdge(v1, v2, log['msg'])

#         graph.add_node(v1)
#         graph.add_node(v2)
#         graph.add_edge(e)
#     return graph


# def visualize_graph(graph, tree_path):
#     print('visualizing graph...')
#     dot = Digraph('IRC Tree', filename=tree_path)
#     dot.graph_attr.update(sep='+100,s100')

#     edges = set()
#     for edge in graph.edges:
#         v1, v2 = edge.src_node, edge.dst_node

#         # comment this block of code to show non-duplicate edges between nodes
#         dot.node(v1.id(), label=v1.label())
#         dot.node(v2.id(), label=v2.label())
#         dot.edge(v1.id(), v2.id())

#         # uncomment this block of code to show duplicate edges between nodes
#         if (v1.id(), v2.id()) not in edges:
#             edges.add((v1.id(), v2.id()))
#             dot.node(v1.id(), label=v1.label())
#             dot.node(v2.id(), label=v2.label())
#             dot.edge(v1.id(), v2.id())
#     dot.view()

### Data loading

In [5]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
log_names_mal = ['03','04','34','39','42','51','56','62']
log_names_benign = ['irc1']
log_names = log_names_mal + log_names_benign

project_dir = '/Users/preneond/Documents/Work/Stratosphere/IRC-Research/IRC-Behavioral-Analysis/'
# project_dir = '/home/prenek/IRC-Behavioral-Analysis/'
log_dir = os.path.join(project_dir, 'zeek/logs/')
out_dir = os.path.join(project_dir, 'python/out/')

# logs_fn_join = [os.path.join(log_dir,l,'irc_join.log') for l in log_names]
# logs_fn_join_mal = [os.path.join(log_dir,l,'irc_join.log') for l in log_names_mal]
# logs_fn_join_benign = [os.path.join(log_dir,l,'irc_join.log') for l in log_names_mal]

logs_fn_privmsg = [os.path.join(log_dir,l,'irc_privmsg.log') for l in log_names]
logs_fn_privmsg_mal = [os.path.join(log_dir,l,'irc_privmsg.log') for l in log_names_mal]
logs_fn_privmsg_benign = [os.path.join(log_dir,l,'irc_privmsg.log') for l in log_names_benign]

In [7]:
from zat.bro_log_reader import BroLogReader
def load_logs(file):
    logs_arr = []
    if not os.path.isfile(file):
        return logs_arr
    
    reader = BroLogReader(file)
    for log in reader.readrows():
        # log is in dictionary format
        logs_arr.append(log)

    return logs_arr

In [8]:
logs_privmsg_mal = [load_logs(l) for l in logs_fn_privmsg_mal]
logs_privmsg_benign = [load_logs(l) for l in logs_fn_privmsg_benign]
logs_privmsg = logs_privmsg_mal + logs_privmsg_benign

Successfully monitoring /Users/preneond/Documents/Work/Stratosphere/IRC-Research/IRC-Behavioral-Analysis/zeek/logs/34/irc_privmsg.log...
Successfully monitoring /Users/preneond/Documents/Work/Stratosphere/IRC-Research/IRC-Behavioral-Analysis/zeek/logs/42/irc_privmsg.log...
Successfully monitoring /Users/preneond/Documents/Work/Stratosphere/IRC-Research/IRC-Behavioral-Analysis/zeek/logs/51/irc_privmsg.log...
Successfully monitoring /Users/preneond/Documents/Work/Stratosphere/IRC-Research/IRC-Behavioral-Analysis/zeek/logs/56/irc_privmsg.log...
Successfully monitoring /Users/preneond/Documents/Work/Stratosphere/IRC-Research/IRC-Behavioral-Analysis/zeek/logs/62/irc_privmsg.log...
Successfully monitoring /Users/preneond/Documents/Work/Stratosphere/IRC-Research/IRC-Behavioral-Analysis/zeek/logs/irc1/irc_privmsg.log...


In [12]:
import networkx as nx

def build_graph(logs):
    print('building graph...')
    graph = nx.DiGraph()
    for log in logs:
        v1_source = log['source']
        v2_target = log['target']
        # v1, v2 = Node(v1_source), Node(v2_target)

        graph.add_node(v1_source)
        graph.add_node(v2_target)
        graph.add_edge(v1_source, v2_target, id=log['msg'])

    return graph

# def visualize_graph(graph, tree_path):
#     print('visualizing graph...')
#     dot = Digraph('IRC Tree', filename=tree_path)
#     dot.graph_attr.update(sep='+100,s100')

#     edges = set()
#     for edge in graph.edges:
#         v1, v2 = edge.src_node, edge.dst_node

#         # comment this block of code to show non-duplicate edges between nodes
#         dot.node(v1.id(), label=v1.label())
#         dot.node(v2.id(), label=v2.label())
#         dot.edge(v1.id(), v2.id())

#         # uncomment this block of code to show duplicate edges between nodes
#         if (v1.id(), v2.id()) not in edges:
#             edges.add((v1.id(), v2.id()))
#             dot.node(v1.id(), label=v1.label())
#             dot.node(v2.id(), label=v2.label())
#             dot.edge(v1.id(), v2.id())
#     dot.view()

In [17]:
import matplotlib.pyplot as plt
# edges_benign = list(map(lambda x: (x['source'], x['target']), logs_privmsg_benign[0]))


# logs_all = []
# for l in logs_privmsg_mal:
#     logs_all += l

# G = build_graph(logs_all)
G = build_graph(logs_privmsg_benign[0])
# visualize_graph(G, 'privmsg_tree_mal')
# pos = n
# pos = nx.fruchterman_reingold_layout(G)
# nx.draw(G, pos)
# nx.draw_spectral(G)
# nx.draw_circular(G)
# nx.draw_kamada_kawai(G)

plt.show()

building graph...


In [20]:
# channel_nodes = list(filter(lambda x: x[0] == '#', G.nodes))
pos = nx.circular_layout(G)
pos_x = {}
pos_y = {}
node_types = {}

for k,v in pos.items():
    pos_x[k] = v[0]
    pos_y[k] = v[1]
    node_types[k] = 'channel' if k[0] == '#' else 'user'

nx.set_node_attributes(G, pos_x, 'x')
nx.set_node_attributes(G, pos_y, 'y')
nx.set_node_attributes(G, node_types, 'type')

nx.write_graphml(G, "graph_privmsg_circular.xml")